In [12]:
import numpy as np
from pygel3d import graph
from pygel3d import hmesh, gl_display as gl
from pygel3d import jupyter_display as jd
import pygel3d as py3d
from plyfile import PlyData, PlyElement
import open3d as o3d
import open3d.core as o3c

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Generate some examples of meshes with previous method to build a mesh from skeleton

### This jupyter note book contains 4 baseline methods
- SQM method
- FEQ remesh method
- Skin Modifier in Blender
- Dual contouring based method

Our original data are `data/points.npy` which contains points' 3d coordinates, `data/edge_array.npy` which contains edge indexed array, `data/edge_radius.npy` which contains each edges radius, corresponding to the edge indexed array(stored by same order).

## Data Preparation

In this section we mainly handle the data. We define several functions to manipulate data.


Load `.npy` document and define a function write the numpy array data into a pygel3d-style `.obj` file. Note: this custom `.obj` file only for graph.

In [5]:
points = np.load('../data/points.npy')
edges = np.load('../data/edge_array.npy')
radii = np.load('../data/edge_radius.npy')

# write a .obj file from the points, edges, and radii numpy arrays
def write_graph_obj(points, edges, radii, path):
    with open(path, 'w') as f:
        # f.write(f'g default\n')
        for point in points:
            f.write(f'n {point[0]} {point[1]} {point[2]}\n')
            
        # for edge in edges:
        #     f.write(f'l {edge[0]+1} {edge[1]+1}\n')

        for i, edge in enumerate(edges):
            radius = radii[i]
            # f.write(f'c {edge[0]} {edge[1]} {radius}\n')
            f.write(f'c {edge[0]} {edge[1]}\n')
            
        f.close()

- Define a function that use the edge radius array generate each vertex's radius, return an np array of vertex radius.
- Define a function that convert the pygel3d-style .obj file to standard .obj file that is readable for Blender.

In [6]:
def edge_radius_to_vertex_radius(edges, edge_radius):
    vertex_radius = np.zeros((edges.max()+1,))
    for i, edge in enumerate(edges):
        vertex_radius[edge[0]] = max(edge_radius[i], vertex_radius[edge[0]])     
        vertex_radius[edge[1]] = max(edge_radius[i], vertex_radius[edge[1]])
    return vertex_radius


def convert_custom_obj_to_standard(input_file, output_file):
    with open(input_file, 'r') as input_obj:
        lines = input_obj.readlines()

    converted_lines = []

    for line in lines:
        if line.startswith('n '):  # Change 'n' to 'v' for vertex
            parts = line.strip().split()
            converted_line = f"v {' '.join(parts[1:])}\n"
            converted_lines.append(converted_line)
        elif line.startswith('c '):  # Change 'c' to 'l' for edges
            parts = line.strip().split()
            converted_line = f"l {' '.join(parts[1:])}\n"
            converted_lines.append(converted_line)

    # Reindex the vertex indices to start from 0
    vertex_index_offset = 0
    for line in converted_lines:
        if line.startswith('l '):
            parts = line.strip().split()
            converted_line = f"l {' '}"
            for v in parts[1:]:
                converted_line += f"{int(v)+1} "
            converted_line += '\n'
            converted_lines[vertex_index_offset] = converted_line
        vertex_index_offset += 1

    with open(output_file, 'w') as output_obj:
        output_obj.writelines(converted_lines)

## SQM method

In [7]:
write_graph_obj(points, edges, radii, "../data/graph.obj")
g = graph.load("../data/graph.obj")
m = graph.to_mesh_cyl(g,1)
hmesh.save('../data/mesh_SQM.obj',m)

## FEQ method

In [10]:
# generate vertex radius from edge radius and save it to vertex_radius.npy
vertex_radius = edge_radius_to_vertex_radius(edges, radii)/10
np.save('vertex_radius.npy', vertex_radius)

# generate feq mesh from vertex radius and save it to feq_mesh.obj
FEQ_mesh = hmesh.skeleton_to_feq(g,vertex_radius)
hmesh.save("../data/feq_mesh.obj",FEQ_mesh)

# generate feq mesh from vertex radius with origin scale and save it to feq_mesh_scale1.obj
FEQ_mesh_t1 = hmesh.skeleton_to_feq(g,vertex_radius*10)
hmesh.save("../data/feq_mesh_scale1.obj",FEQ_mesh_t1)

## Duel contouring based method

- Generate mesh only using points information

In [14]:
# create numpy array of shape (n, n, n) from list of indices
def voxel_grid_from_idx(indices):
    voxel_grid = np.zeros((indices.max(axis=0)+1))
    for index in indices:
        voxel_grid[tuple(index)] = 1
    return voxel_grid

In [13]:
# generate point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
o3d.io.write_point_cloud("../data/pcd.ply", pcd)

# load point cloud and create voxel grid
pcd_load = o3d.io.read_point_cloud("../data/pcd.ply")
pcd_load.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(len(points), 3)))
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_load, voxel_size=5)

voxels = voxel_grid.get_voxels()
indices = np.stack(list(vx.grid_index for vx in voxels))
print(indices)

voxel_grid = voxel_grid_from_idx(indices)
iso_mesh = 0
iso_mesh = hmesh.volumetric_isocontour(voxel_grid,high_is_inside=True,make_triangles=True,tau=0.85)
hmesh.save('../data/iso_mesh.obj', iso_mesh)


[[ 46  74 150]
 [ 60   4  56]
 [ 61  21 143]
 ...
 [ 13  34 118]
 [ 16  70  55]
 [ 84  22  47]]


- First generate voxel grid from SQM mesh, then create the mesh from voxel grid.

In [17]:
stick_mesh = hmesh.load("../data/mesh_SQM.obj")
hmesh.triangulate(stick_mesh)
hmesh.save("../data/mesh_SQM_tri.obj", stick_mesh)


In [18]:
stick_mesh = o3d.io.read_triangle_mesh("../data/mesh_SQM_tri.obj")
voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(stick_mesh, 4)
o3d.io.write_voxel_grid("../data/voxel_grid.ply", voxel_grid)
voxels = voxel_grid.get_voxels()
indices = np.stack(list(vx.grid_index for vx in voxels))
print(indices)


voxel_grid = voxel_grid_from_idx(indices)
iso_mesh = 0
iso_mesh = hmesh.volumetric_isocontour(voxel_grid,high_is_inside=True,make_triangles=True,tau=0.85)
hmesh.save('../data/iso_mesh_from_stick.obj', iso_mesh)

[[ 53  39 221]
 [126  31  64]
 [ 71  70  32]
 ...
 [ 33  16  90]
 [ 59  13 130]
 [ 59  14 130]]


In [10]:
# skeleton = graph.LS_skeleton(g)
# graph.save('../data/skeleton.obj', skeleton)
# convert_custom_obj_to_standard('../data/skeleton.obj', '../data/skeleton_standard.obj')